# 🧠 STAMP Demo — Tile Embedding Extraction from WSIs
This Colab notebook demonstrates how to use **[KatherLab's STAMP](https://github.com/KatherLab/STAMP)** to extract **tile-level embeddings** from Whole Slide Images (WSIs) using a pretrained model (Virchow2).

### 📋 Steps
1. Environment Setup (GPU, libraries)
2. Download a demo WSI
3. Preprocess (tiling)
4. Encode (extract embeddings)
5. Inspect and visualize results

⚙️ **Output:** Tile embeddings (`.h5` files) and tiles saved to Google Drive.

## 1️⃣ Environment Setup

In [ ]:
!nvidia-smi
!apt-get update -y
!apt-get install -y libgl1-mesa-glx
!pip install "git+https://github.com/KatherLab/STAMP.git[gpu]" h5py matplotlib pillow openslide-python

from google.colab import drive
drive.mount('/content/drive')

import os, matplotlib.pyplot as plt, h5py, numpy as np
os.makedirs("/content/drive/MyDrive/STAMP_demo_features", exist_ok=True)

## 2️⃣ Download Demo WSI

In [ ]:
!wget -q https://github.com/PathologyData/sample-wsi/raw/main/demo_slide.svs -O demo_slide.svs
!ls -lh demo_slide.svs

## 3️⃣ Create Config File

In [ ]:
%%bash
cat > config.yaml <<EOF
data:
  slides_dir: "/content"
  metadata_csv: null
  output_dir: "/content/drive/MyDrive/STAMP_demo_features"

preprocess:
  tile_size: 512
  magnification: 20
  overlap: 0
  num_workers: 2

encode:
  backbone: virchow2
  batch_size: 8
  save_format: h5
  gpu: true
EOF

!cat config.yaml

## 4️⃣ Preprocess (Extract Tiles)

In [ ]:
!stamp preprocess -c config.yaml

## 5️⃣ Encode Slides → Tile Embeddings

In [ ]:
!stamp encode_slides -c config.yaml

## 6️⃣ Inspect Results

In [ ]:
import glob
embed_files = glob.glob("/content/drive/MyDrive/STAMP_demo_features/embeddings/*.h5")
print("Found embedding files:", embed_files)

if embed_files:
    with h5py.File(embed_files[0], 'r') as f:
        print("Available keys:", list(f.keys()))
        for key in f.keys():
            data = f[key][:]
            print(f"{key}: shape={data.shape}, dtype={data.dtype}")
            if data.ndim == 2:
                print("Example vector:", data[0][:10])

## 7️⃣ Visualize Random Tiles

In [ ]:
import glob
from PIL import Image
tile_files = sorted(glob.glob("/content/drive/MyDrive/STAMP_demo_features/tiles/**/*.png", recursive=True))
print(f"Total tiles: {len(tile_files)}")
if tile_files:
    fig, axes = plt.subplots(1, min(5,len(tile_files)), figsize=(15,3))
    for ax, tile in zip(axes, tile_files[:5]):
        ax.imshow(Image.open(tile))
        ax.axis('off')
    plt.suptitle("Example tiles from demo WSI")
    plt.show()

print("✅ Done — embeddings saved in /content/drive/MyDrive/STAMP_demo_features/")